In [1]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
import pandas as pd
from Bio import SeqIO
import json
import os


def get_gap_mismatch_pairwise2_alignment(seqA, seqB):

    alignments = pairwise2.align.globalxx(seqA, seqB)

    # Print the alignment
    # print(format_alignment(*alignments[0]))

    # Extract the aligned sequences
    aligned_A = alignments[0][0]
    aligned_B = alignments[0][1]

    # Initialize counters for gaps and mismatches
    gap_count = 0
    mismatch_count = 0

    # Iterate through the aligned sequences to count gaps and mismatches
    for a, b in zip(aligned_A, aligned_B):
        if a == '-' or b == '-':
            gap_count += 1
        elif a != b:
            mismatch_count += 1

    # print("Gap Count:", gap_count)
    # print("Mismatch Count:", mismatch_count)

    n_count = str(seqB).count('N')

    return {"score": alignments[0].score, "gaps": gap_count, "mismatches": mismatch_count, "N_count": n_count}


def get_scores_for_sample(sample_consensus, mbs_consensus, mbs_consensus_name):
    
    score_metrics_dict = {}

    score_all_fragments = 0
    gaps_all_fragments = 0
    mismatch_all_fragments = 0
    n_count_all_fragments = 0

    # fragmenty pre vzorku
    for sample_fragment in sample_consensus.keys():
        
        fragment = sample_fragment.split('_')[-1] # skratka fragmentu genu

        # sekvencia (fragmentu) vzorky
        sample_consensus_seq = sample_consensus[sample_fragment].seq 
        
        # sekvencia (fragmentu) mbs
        mbs_consensus_fragment = mbs_consensus_name + fragment
        mbs_consensus_seq = mbs_consensus[mbs_consensus_fragment].seq
        
        # prorovnanie consenzov (po fragmentoch), out: dict{score, gap, mismatch}; ulozenie do dict na poziciu fragmentu
        score_metrics_dict[fragment]  = get_gap_mismatch_pairwise2_alignment(mbs_consensus_seq, sample_consensus_seq)

        # cout scores for all fragments
        score_all_fragments += score_metrics_dict[fragment]['score']
        gaps_all_fragments += score_metrics_dict[fragment]['gaps']
        mismatch_all_fragments += score_metrics_dict[fragment]['mismatches']
        n_count_all_fragments += score_metrics_dict[fragment]['N_count']

    # pridaj parameter pre vsetky fragmenty (score = count cez vsetky fragmenty)
    score_metrics_dict['all_fragments'] = {'score': score_all_fragments, 'gaps': gaps_all_fragments, 'mismatches': mismatch_all_fragments, 'N_count':  n_count_all_fragments}

    return score_metrics_dict
        

# Porovnanie consenzov mbs_consenzus vs sample_consenzus

# mbs_consensus_names: pre kazdy virus nazov(id)
mbs_consensus_names = {'h1n1': 'run230320_UVZ_BA_23-vsp-1337_A__WISCONSIN__588__2019__H1N1_',
                       'h3n2': 'run230320_UVZ_BA_23-vsp-2129_A__DARWIN__6__2021__H3N2_',
                       'rsv_b': 'run230320_UVZ_BA_23-vsp-902_hRSV__B__Australia__VIC-', 
                       # 'sars_cov_2': '' 
                       'victoria': 'run230320_UVZ_BA_23-vsp-2036_B__AUSTRIA__1359417__2021__VICTORIA_'
                       }

samples_names = {'h1n1': 'run230403_UVZ_BA_23-vsp-3269_',
                 'h3n2': 'run221223_UVZ_BA_22-vsp-42063_',
                 'rsv_b': 'run230403_UVZ_BA_23-vsp-3484_',
                 'sars_cov_2': 'run230403_UVZ_BA_23-vsp-3341_',
                 'victoria': 'run230522_UVZ_BA_23-vsp-4318_'
                 }

parameteres_df = pd.read_csv('./data/parameters_set.tsv', sep='\t')

samples_score_metrics_dict = {}

# vzorka h1n1
virus = 'h1n1'
consensus_baseline_path = "./consensus_baseline/1337_consensus-h1n1_2019-wgs.fa"

# nazov vzorky
for sample_id in [samples_names[virus]]:
    
    samples_score_metrics_parameter_dict = {}

    # cez vyber nastaveni respiracnej pipeliny
    for parameter in list(parameteres_df[:55]['parameters']):
        
        sample_name = sample_id + parameter
    
        consensus_sample_path = "./data/report_pollux/" + sample_name + "/_summary/consensus-h1n1_2019-wgs.fa"

        if os.path.exists(consensus_sample_path):

            mbs_consensus_name = mbs_consensus_names[virus]
            mbs_consensus = SeqIO.to_dict(SeqIO.parse(consensus_baseline_path, "fasta"))
            sample_consensus = SeqIO.to_dict(SeqIO.parse(consensus_sample_path, "fasta"))

            samples_score_metrics_parameter_dict[parameter] = get_scores_for_sample(sample_consensus, mbs_consensus, mbs_consensus_name)

    samples_score_metrics_dict[sample_id] = samples_score_metrics_parameter_dict

samples_score_metrics_dict


{'run230403_UVZ_BA_23-vsp-3269_': {}}

41

In [30]:
from Bio import Align


def get_scores_for_sample_aligner_set_scores(sample_consensus, mbs_consensus, mbs_consensus_name):

    score_metrics_dict = {}

    score_all_fragments = 0
    match_all_fragments = 0
    gaps_all_fragments = 0
    mismatch_all_fragments = 0
    n_count_all_fragments = 0

    # fragmenty pre vzorku
    for sample_fragment in sample_consensus.keys():
        
        fragment = sample_fragment.split('_')[-1] # skratka fragmentu genu

        # sekvencia (fragmentu) vzorky
        sample_consensus_seq = sample_consensus[sample_fragment].seq 
        
        # sekvencia (fragmentu) mbs
        mbs_consensus_fragment = mbs_consensus_name + fragment
        mbs_consensus_seq = mbs_consensus[mbs_consensus_fragment].seq
        
        # prorovnanie consenzov (po fragmentoch), out: dict{score, gap, mismatch}; ulozenie do dict na poziciu fragmentu
        # score_metrics_dict[fragment]  = get_gap_mismatch_pairwise2_alignment(mbs_consensus_seq, sample_consensus_seq)

        aligner = Align.PairwiseAligner()
        aligner.mode = "global" 
        aligner.match_score = 2
        aligner.mismatch_score = -1
        aligner.open_gap_score = -5
        aligner.extend_gap_score = -1

        alignment = aligner.align(mbs_consensus_seq, sample_consensus_seq)

        for align in alignment:
            align_str = str(align)
        
        print(aligner.match_score, align_str.count('|'), aligner.mismatch_score, align_str.count('.'))
        score_metrics_dict[fragment] = {'score': align_str.count('|'), 'match': align_str.count('|'), 'gaps': align_str.count('-'), 'mismatches': align_str.count('.'), 'N_count': align_str.count('N')}

        # cout scores for all fragments
        score_all_fragments += score_metrics_dict[fragment]['score']
        match_all_fragments += score_metrics_dict[fragment]['match']
        gaps_all_fragments += score_metrics_dict[fragment]['gaps']
        mismatch_all_fragments += score_metrics_dict[fragment]['mismatches']
        n_count_all_fragments += score_metrics_dict[fragment]['N_count']

    # pridaj parameter pre vsetky fragmenty (score = count cez vsetky fragmenty)
    score_metrics_dict['all_fragments'] = {'score': score_all_fragments, 'match': match_all_fragments, 'gaps': gaps_all_fragments, 'mismatches': mismatch_all_fragments, 'N_count':  n_count_all_fragments}

    return score_metrics_dict


In [7]:
samples_names = {'h1n1': 'run230403_UVZ_BA_23-vsp-3337_',
 'h3n2': 'run221223_UVZ_BA_22-vsp-42063_',
 'rsv_b': 'run230403_UVZ_BA_23-vsp-3484_',
 'sars_cov_2': 'run230403_UVZ_BA_23-vsp-3341_',
 'victoria': 'run230522_UVZ_BA_23-vsp-4318_'}

In [31]:
# Zarovnanie pomocou Align.PairwiseAligner
from Bio import Align


samples_score_metrics_dict_aligner_set_scores = {}

# print('zarovanie s def skore pre g, mism, match, ...')
for sample_id in [samples_names[virus]]:
    
    samples_score_metrics_parameter_dict_aligner_set_scores = {}

    # cez vyber nastaveni respiracnej pipeliny
    for parameter in list(parameteres_df['parameters']):
        
        sample_name = sample_id + parameter
    
        consensus_sample_path = "./data/report_pollux/" + sample_name + "/_summary/consensus-h1n1_2019-wgs.fa"

        if os.path.exists(consensus_sample_path):

            mbs_consensus_name = mbs_consensus_names[virus]
            mbs_consensus = SeqIO.to_dict(SeqIO.parse(consensus_baseline_path, "fasta"))
            sample_consensus = SeqIO.to_dict(SeqIO.parse(consensus_sample_path, "fasta"))

            samples_score_metrics_parameter_dict_aligner_set_scores[parameter] = get_scores_for_sample_aligner_set_scores(sample_consensus, mbs_consensus, mbs_consensus_name)

    samples_score_metrics_dict_aligner_set_scores[sample_id] = samples_score_metrics_parameter_dict_aligner_set_scores



2.0 2276 -1.0 40
2.0 2296 -1.0 20
2.0 2190 -1.0 18
2.0 1728 -1.0 24
2.0 1506 -1.0 34
2.0 1415 -1.0 18
2.0 991 -1.0 11
2.0 858 -1.0 7
2.0 2276 -1.0 40
2.0 2296 -1.0 20
2.0 2190 -1.0 18
2.0 1728 -1.0 24
2.0 1506 -1.0 34
2.0 1415 -1.0 18
2.0 991 -1.0 11
2.0 858 -1.0 7
2.0 2276 -1.0 40
2.0 2296 -1.0 20
2.0 2190 -1.0 18
2.0 1728 -1.0 24
2.0 1506 -1.0 34
2.0 1415 -1.0 18
2.0 991 -1.0 11
2.0 858 -1.0 7
2.0 2276 -1.0 40
2.0 2296 -1.0 20
2.0 2190 -1.0 18
2.0 1728 -1.0 24
2.0 1506 -1.0 34
2.0 1415 -1.0 18
2.0 991 -1.0 11
2.0 858 -1.0 7
2.0 2276 -1.0 40
2.0 2296 -1.0 20
2.0 2190 -1.0 18
2.0 1728 -1.0 24
2.0 1506 -1.0 34
2.0 1415 -1.0 18
2.0 991 -1.0 11
2.0 858 -1.0 7
2.0 2276 -1.0 40
2.0 2296 -1.0 20
2.0 2190 -1.0 18
2.0 1728 -1.0 24
2.0 1506 -1.0 34
2.0 1415 -1.0 18
2.0 991 -1.0 11
2.0 858 -1.0 7
2.0 2276 -1.0 40
2.0 2296 -1.0 20
2.0 2190 -1.0 18
2.0 1728 -1.0 24
2.0 1506 -1.0 34
2.0 1416 -1.0 17
2.0 991 -1.0 11
2.0 858 -1.0 7
2.0 2276 -1.0 40
2.0 2296 -1.0 20
2.0 2190 -1.0 18
2.0 1728 -1.0 24
2

In [14]:
# json pre 3337
samples_score_metrics_dict_aligner_set_scores['run230403_UVZ_BA_23-vsp-3337_'].keys()

56

In [15]:
# uloz do jsonu
path_to_save = './data/result_metrics/sample_' + samples_names[virus] + '_score_metrics_PairwiseAligner.json'

with open(path_to_save, 'w') as fp:
    json.dump(samples_score_metrics_dict_aligner_set_scores, fp)

In [32]:
import numpy as np

samples_score_metrics_parameter_dict = samples_score_metrics_dict_aligner_set_scores['run230403_UVZ_BA_23-vsp-3337_']
# vytvor dataframe z jsonu
scores_colums = ['parameters', 'score', 'gap', 'mismatch', 'N_count', 'score_agg']
samples_score_metrics_parameter_df = pd.DataFrame(columns=scores_colums)

sample_name = [samples_names[virus]]

for parameter in list(samples_score_metrics_parameter_dict.keys()):  # samples_score_metrics_dict[sample]
    score =  samples_score_metrics_parameter_dict[parameter]['all_fragments']['score']
    gap = samples_score_metrics_parameter_dict[parameter]['all_fragments']['gaps']
    mismatch = samples_score_metrics_parameter_dict[parameter]['all_fragments']['mismatches']
    n_count = samples_score_metrics_parameter_dict[parameter]['all_fragments']['N_count']
    score_agg = score - gap - mismatch

    params_df = pd.DataFrame([[parameter, score, gap, mismatch, n_count, score_agg]], columns=scores_colums)

    samples_score_metrics_parameter_df = pd.concat([samples_score_metrics_parameter_df, params_df], axis=0, ignore_index=True)


# rozel 'parameter name' na viacero stlpcov
col_names_param = ['trimmer', 'quality', 'headcrop', 'minlen', 'score', 'gap', 'mismatch', 'N_count', 'score_agg']
scores_df = pd.DataFrame(columns=col_names_param)

for _, row in samples_score_metrics_parameter_df.iterrows():
    
    row_scores_df = pd.DataFrame([[np.nan, np.nan, np.nan, np.nan, row['score'], row['gap'], row['mismatch'], row['N_count'], row['score_agg']]],
                                   columns=col_names_param)
    param_name = row['parameters']
    
    row_scores_df['trimmer'] = param_name.split('_')[0]
    row_scores_df['quality'] = param_name.split('_')[1].replace('tripq', '')
    row_scores_df['headcrop'] = param_name.split('_')[2].replace('headcrop', '')
    row_scores_df['minlen'] = param_name.split('_')[3].replace('minlen', '')

    scores_df = pd.concat([scores_df, row_scores_df], axis=0, ignore_index=True)

scores_df

# uloz do suboru
# path_to_save = './data/result_metrics/sample_' + samples_names[virus] + 'score_metrics_PairwiseAligner.tsv'
# scores_df.to_csv(path_to_save, sep='\t', index=False)

trimmer quality headcrop minlen  score gap mismatch N_count score_agg
0   trimmomatic      10        0     35  13260   0      172       9     13088
1   trimmomatic      10        0     45  13260   0      172       9     13088
2   trimmomatic      10        0     55  13260   0      172       9     13088
3   trimmomatic      10        0     65  13260   0      172       9     13088
4   trimmomatic      10        5     35  13260   0      172       9     13088
5   trimmomatic      10        5     45  13260   0      172       9     13088
6   trimmomatic      10        5     55  13261   0      171      10     13090
7   trimmomatic      10        5     65  13261   0      171      10     13090
8   trimmomatic      10       10     35  13252   0      180      19     13072
9   trimmomatic      10       10     45  13252   0      180      19     13072
10  trimmomatic      10       10     55  13252   0      180      19     13072
11  trimmomatic      10       10     65  13252   0      180      19     13072
12  trimmomatic      10       15     35  13236   0      196      35     13040
13  trimmomatic      10       15     45  13235   0      197      36     13038
14  trimmomatic      10       15     55  13234   0      198      37     13036
15  trimmomatic      10       15     65  13234   0      198      37     13036
16  trimmomatic      15        0     35  13260   0      172       9     13088
17  trimmomatic      15        0     45  13260   0      172       9     13088
18  trimmomatic      15        0     55  13260   0      172       9     13088
19  trimmomatic      15        0     65  13260   0      172       9     13088
20  trimmomatic      15        5     35  13260   0      172       9     13088
21  trimmomatic      15        5     45  13260   0      172       9     13088
22  trimmomatic      15        5     55  13261   0      171      10     13090
23  trimmomatic      15        5     65  13261   0      171      10     13090
24  trimmomatic      15       10     35  13252   0      180      19     13072
25  trimmomatic      15       10     45  13252   0      180      19     13072
26  trimmomatic      15       10     55  13252   0      180      19     13072
27  trimmomatic      15       10     65  13251   0      181      20     13070
28  trimmomatic      15       15     35  13235   0      197      36     13038
29  trimmomatic      15       15     45  13235   0      197      36     13038
30  trimmomatic      15       15     55  13234   0      198      37     13036
31  trimmomatic      15       15     65  13232   0      200      39     13032
32  trimmomatic      20        0     35  13260   0      172       9     13088
33  trimmomatic      20        0     45  13260   0      172       9     13088
34  trimmomatic      20        0     55  13260   0      172       9     13088
35  trimmomatic      20        0     65  13260   0      172       9     13088
36  trimmomatic      20        5     35  13260   0      172      11     13088
37  trimmomatic      20        5     45  13260   0      172      11     13088
38  trimmomatic      20        5     55  13260   0      172      11     13088
39  trimmomatic      20        5     65  13260   0      172      11     13088
40  trimmomatic      20       10     35  13251   0      181      20     13070
41  trimmomatic      20       10     45  13251   0      181      20     13070
42  trimmomatic      20       10     55  13251   0      181      20     13070
43  trimmomatic      20       10     65  13251   0      181      20     13070
44  trimmomatic      20       15     35  13235   0      197      36     13038
45  trimmomatic      20       15     45  13234   0      198      37     13036
46  trimmomatic      20       15     55  13233   0      199      38     13034
47  trimmomatic      20       15     65  13232   0      200      39     13032
48  trimmomatic      25        0     35  13260   0      172       9     13088
49  trimmomatic      25        0     45  13260   0      172       9     13088
50  trimmomatic      25       

In [34]:
path_to_save = './data/result_metrics/sample_' + samples_names[virus] + 'score_metrics_PairwiseAligner.tsv'
scores_df.sort_values(by='score', ascending=False).to_csv(path_to_save, sep='\t', index=False)


In [15]:
len(list(samples_score_metrics_dict_aligner_set_scores['run230403_UVZ_BA_23-vsp-3269_'].keys()))

56

In [17]:
samples_score_metrics_parameter_dict_aligner_set_scores.keys()

dict_keys(['trimmomatic_tripq10_headcrop0_minlen35', 'trimmomatic_tripq10_headcrop0_minlen45', 'trimmomatic_tripq10_headcrop0_minlen55', 'trimmomatic_tripq10_headcrop0_minlen65', 'trimmomatic_tripq10_headcrop5_minlen35', 'trimmomatic_tripq10_headcrop5_minlen45', 'trimmomatic_tripq10_headcrop5_minlen65', 'trimmomatic_tripq10_headcrop10_minlen35', 'trimmomatic_tripq10_headcrop10_minlen45', 'trimmomatic_tripq10_headcrop10_minlen65', 'trimmomatic_tripq10_headcrop15_minlen45', 'trimmomatic_tripq10_headcrop15_minlen55', 'trimmomatic_tripq10_headcrop15_minlen65', 'trimmomatic_tripq15_headcrop0_minlen35', 'trimmomatic_tripq15_headcrop0_minlen45', 'trimmomatic_tripq15_headcrop0_minlen55', 'trimmomatic_tripq15_headcrop0_minlen65', 'trimmomatic_tripq15_headcrop5_minlen35', 'trimmomatic_tripq15_headcrop5_minlen45', 'trimmomatic_tripq15_headcrop5_minlen65', 'trimmomatic_tripq15_headcrop10_minlen45', 'trimmomatic_tripq15_headcrop15_minlen45', 'trimmomatic_tripq15_headcrop15_minlen55', 'trimmomatic_t

In [20]:
scores_colums = ['parameters', 'score', 'match', 'gap', 'mismatch', 'N_count'] #, 'score_agg']
samples_score_metrics_parameter_df = pd.DataFrame(columns=scores_colums)

# for sample in list(samples):
for parameter in list(samples_score_metrics_dict_aligner_set_scores['run230403_UVZ_BA_23-vsp-3269_'].keys()):  # samples_score_metrics_dict[sample]
    score =  samples_score_metrics_parameter_dict_aligner_set_scores[parameter]['all_fragments']['score']
    match = samples_score_metrics_parameter_dict_aligner_set_scores[parameter]['all_fragments']['match']
    gap = samples_score_metrics_parameter_dict_aligner_set_scores[parameter]['all_fragments']['gaps']
    mismatch = samples_score_metrics_parameter_dict_aligner_set_scores[parameter]['all_fragments']['mismatches']
    n_count = samples_score_metrics_parameter_dict_aligner_set_scores[parameter]['all_fragments']['N_count']
    # score_agg = score - gap - mismatch

    params_df = pd.DataFrame([[parameter, score, match, gap, mismatch, n_count]], columns=scores_colums)

    samples_score_metrics_parameter_df = pd.concat([samples_score_metrics_parameter_df, params_df], axis=0, ignore_index=True)

samples_score_metrics_parameter_df

parameters  score  match gap mismatch N_count
0    trimmomatic_tripq10_headcrop0_minlen35   16.0  12932   0      500     479
1    trimmomatic_tripq10_headcrop0_minlen45   16.0  12932   0      500     479
2    trimmomatic_tripq10_headcrop0_minlen55   16.0  12932   0      500     479
3    trimmomatic_tripq10_headcrop0_minlen65   16.0  12932   0      500     479
4    trimmomatic_tripq10_headcrop5_minlen35   16.0  12932   0      500     479
5    trimmomatic_tripq10_headcrop5_minlen45   16.0  12932   0      500     479
6    trimmomatic_tripq10_headcrop5_minlen65   16.0  12932   0      500     479
7   trimmomatic_tripq10_headcrop10_minlen35   16.0  12932   0      500     479
8   trimmomatic_tripq10_headcrop10_minlen45   16.0  12932   0      500     479
9   trimmomatic_tripq10_headcrop10_minlen65   16.0  12932   0      500     479
10  trimmomatic_tripq10_headcrop15_minlen45   16.0  12932   0      500     479
11  trimmomatic_tripq10_headcrop15_minlen55   16.0  12932   0      500     479
12  trimmomatic_tripq10_headcrop15_minlen65   16.0  12932   0      500     479
13   trimmomatic_tripq15_headcrop0_minlen35   16.0  12931   0      501     480
14   trimmomatic_tripq15_headcrop0_minlen45   16.0  12931   0      501     480
15   trimmomatic_tripq15_headcrop0_minlen55   16.0  12931   0      501     480
16   trimmomatic_tripq15_headcrop0_minlen65   16.0  12931   0      501     480
17   trimmomatic_tripq15_headcrop5_minlen35   16.0  12931   0      501     480
18   trimmomatic_tripq15_headcrop5_minlen45   16.0  12931   0      501     480
19   trimmomatic_tripq15_headcrop5_minlen65   16.0  12931   0      501     480
20  trimmomatic_tripq15_headcrop10_minlen45   16.0  12931   0      501     480
21  trimmomatic_tripq15_headcrop15_minlen45   16.0  12931   0      501     480
22  trimmomatic_tripq15_headcrop15_minlen55   16.0  12931   0      501     480
23   trimmomatic_tripq20_headcrop0_minlen35   16.0  12921   0      511     490
24   trimmomatic_tripq20_headcrop0_minlen45   16.0  12921   0      511     490
25   trimmomatic_tripq20_headcrop0_minlen65   16.0  12921   0      511     490
26   trimmomatic_tripq20_headcrop5_minlen35   16.0  12921   0      511     490
27   trimmomatic_tripq20_headcrop5_minlen45   16.0  12921   0      511     490
28   trimmomatic_tripq20_headcrop5_minlen65   16.0  12921   0      511     490
29  trimmomatic_tripq20_headcrop10_minlen45   16.0  12921   0      511     490
30  trimmomatic_tripq20_headcrop10_minlen55   16.0  12921   0      511     490
31  trimmomatic_tripq20_headcrop15_minlen35   16.0  12921   0      511     490
32  trimmomatic_tripq20_headcrop15_minlen45   16.0  12921   0      511     490
33  trimmomatic_tripq20_headcrop15_minlen55   16.0  12921   0      511     490
34  trimmomatic_tripq20_headcrop15_minlen65   16.0  12921   0      511     490
35   trimmomatic_tripq25_headcrop0_minlen35   16.0  12907   0      525     504
36   trimmomatic_tripq25_headcrop0_minlen45   16.0  12907   0      525     504
37   trimmomatic_tripq25_headcrop0_minlen55   16.0  12907   0      525     504
38   trimmomatic_tripq25_headcrop0_minlen65   16.0  12907   0      525     504
39   trimmomatic_tripq25_headcrop5_minlen35   16.0  12907   0      525     504
40   trimmomatic_tripq25_headcrop5_minlen55   16.0  12907   0      525     504
41   trimmomatic_tripq25_headcrop5_minlen65   16.0  12907   0      525     504
42  trimmomatic_tripq25_headcrop10_minlen35   16.0  12907   0      525     504
43  trimmomatic_tripq25_headcrop10_minlen45   16.0  12907   0      525     504
44  trimmomatic_tripq25_headcrop10_minlen55   16.0  12907   0      525     504
45  trimmomatic_tripq25_headcrop10_minlen65   16.0  12907   0      525     504
46  trimmomatic_tripq25_headcrop15_minlen35   16.0  12907   0      525     504
47  trimmomatic_tripq25_headcrop15_minlen45   16.0  12907   0      525     504
48  trimmomatic_tripq25_headcrop15_minlen55   16.0  12907   0      525     504
49  trimmomatic_tripq25_headcrop15_minlen65   16.0  12907   0      525     504
50  

In [21]:
import numpy as np

col_names_param = ['trimmer', 'headcrop', 'quality', 'score', 'match', 'gap', 'mismatch', 'N_count']

scores_df = pd.DataFrame(columns=col_names_param)

for _, row in samples_score_metrics_parameter_df.iterrows():
    
    row_scores_df = pd.DataFrame([[np.nan, np.nan, np.nan, row['score'], row['match'], row['gap'], row['mismatch'], row['N_count']]],
                                   columns=col_names_param)
    param_name = row['parameters']
    
    row_scores_df['trimmer'] = param_name.split('_')[0]
    row_scores_df['headcrop'] = param_name.split('_')[1].replace('tripq', '')
    row_scores_df['quality'] = param_name.split('_')[2].replace('headcrop', '')

    scores_df = pd.concat([scores_df, row_scores_df], axis=0, ignore_index=True)

scores_df


trimmer headcrop quality  score  match gap mismatch N_count
0   trimmomatic       10       0   16.0  12932   0      500     479
1   trimmomatic       10       0   16.0  12932   0      500     479
2   trimmomatic       10       0   16.0  12932   0      500     479
3   trimmomatic       10       0   16.0  12932   0      500     479
4   trimmomatic       10       5   16.0  12932   0      500     479
5   trimmomatic       10       5   16.0  12932   0      500     479
6   trimmomatic       10       5   16.0  12932   0      500     479
7   trimmomatic       10      10   16.0  12932   0      500     479
8   trimmomatic       10      10   16.0  12932   0      500     479
9   trimmomatic       10      10   16.0  12932   0      500     479
10  trimmomatic       10      15   16.0  12932   0      500     479
11  trimmomatic       10      15   16.0  12932   0      500     479
12  trimmomatic       10      15   16.0  12932   0      500     479
13  trimmomatic       15       0   16.0  12931   0      501     480
14  trimmomatic       15       0   16.0  12931   0      501     480
15  trimmomatic       15       0   16.0  12931   0      501     480
16  trimmomatic       15       0   16.0  12931   0      501     480
17  trimmomatic       15       5   16.0  12931   0      501     480
18  trimmomatic       15       5   16.0  12931   0      501     480
19  trimmomatic       15       5   16.0  12931   0      501     480
20  trimmomatic       15      10   16.0  12931   0      501     480
21  trimmomatic       15      15   16.0  12931   0      501     480
22  trimmomatic       15      15   16.0  12931   0      501     480
23  trimmomatic       20       0   16.0  12921   0      511     490
24  trimmomatic       20       0   16.0  12921   0      511     490
25  trimmomatic       20       0   16.0  12921   0      511     490
26  trimmomatic       20       5   16.0  12921   0      511     490
27  trimmomatic       20       5   16.0  12921   0      511     490
28  trimmomatic       20       5   16.0  12921   0      511     490
29  trimmomatic       20      10   16.0  12921   0      511     490
30  trimmomatic       20      10   16.0  12921   0      511     490
31  trimmomatic       20      15   16.0  12921   0      511     490
32  trimmomatic       20      15   16.0  12921   0      511     490
33  trimmomatic       20      15   16.0  12921   0      511     490
34  trimmomatic       20      15   16.0  12921   0      511     490
35  trimmomatic       25       0   16.0  12907   0      525     504
36  trimmomatic       25       0   16.0  12907   0      525     504
37  trimmomatic       25       0   16.0  12907   0      525     504
38  trimmomatic       25       0   16.0  12907   0      525     504
39  trimmomatic       25       5   16.0  12907   0      525     504
40  trimmomatic       25       5   16.0  12907   0      525     504
41  trimmomatic       25       5   16.0  12907   0      525     504
42  trimmomatic       25      10   16.0  12907   0      525     504
43  trimmomatic       25      10   16.0  12907   0      525     504
44  trimmomatic       25      10   16.0  12907   0      525     504
45  trimmomatic       25      10   16.0  12907   0      525     504
46  trimmomatic       25      15   16.0  12907   0      525     504
47  trimmomatic       25      15   16.0  12907   0      525     504
48  trimmomatic       25      15   16.0  12907   0      525     504
49  trimmomatic       25      15   16.0  12907   0      525     504
50  trimmomatic       30       0   16.0  12281   0     1151    1132
51  trimmomatic       30       0   16.0  12281   0     1151    1132
52  trimmomatic       30       0   16.0  12281   0     1151    1132
53  trimmomatic       30       0   16.0  12281   0     1151    1132
54  trimmomatic       30       5   16.0  12281   0     1151    1132
55  trimmomatic       30       5   16.0  12281   0     1151    1132

In [22]:
scores_df.to_csv('./data/result_metrics/sample_run230403_UVZ_BA_23-vsp-3269_pairwiseAligner_set_scores.tsv', sep='\t', index=False)


In [14]:
len(mbs_consensus['run230320_UVZ_BA_23-vsp-1337_A__WISCONSIN__588__2019__H1N1_PB2'].seq)

2316

In [ ]:
# substitution_matrices

In [4]:
from Bio.Align import substitution_matrices
from Bio import Align


def get_scores_for_sample_substitution_matrix(sample_consensus, mbs_consensus, mbs_consensus_name):

    score_metrics_dict = {}

    score_all_fragments = 0
    match_all_fragments = 0
    gaps_all_fragments = 0
    mismatch_all_fragments = 0
    n_count_all_fragments = 0

    # fragmenty pre vzorku
    for sample_fragment in sample_consensus.keys():
        
        fragment = sample_fragment.split('_')[-1] # skratka fragmentu genu

        # sekvencia (fragmentu) vzorky
        sample_consensus_seq = sample_consensus[sample_fragment].seq 
        
        # sekvencia (fragmentu) mbs
        mbs_consensus_fragment = mbs_consensus_name + fragment
        mbs_consensus_seq = mbs_consensus[mbs_consensus_fragment].seq
        
        # prorovnanie consenzov (po fragmentoch), out: dict{score, gap, mismatch}; ulozenie do dict na poziciu fragmentu
        # score_metrics_dict[fragment]  = get_gap_mismatch_pairwise2_alignment(mbs_consensus_seq, sample_consensus_seq)

        aligner = Align.PairwiseAligner()
        aligner.mode = "global" 
        aligner.match_score = 2
        aligner.mismatch_score = -1
        aligner.open_gap_score = -5
        aligner.extend_gap_score = -1

        substitution_matrix = substitution_matrices.load("BLOSUM62")
        aligner.substitution_matrix = substitution_matrix

        alignment = aligner.align(mbs_consensus_seq, sample_consensus_seq)

        for align in alignment:
            align_str = str(align)
        
        # print(alignment.score, align.score,  align_str.count('|'), align_str.count('-'), align_str.count('.'), align_str.count('N'))

        score_metrics_dict[fragment] = {'score': alignment.score, 'match': align_str.count('|'), 'gaps': align_str.count('-'), 'mismatches': align_str.count('.'), 'N_count': align_str.count('N')}

        # cout scores for all fragments
        score_all_fragments += score_metrics_dict[fragment]['score']
        match_all_fragments += score_metrics_dict[fragment]['match']        
        gaps_all_fragments += score_metrics_dict[fragment]['gaps']
        mismatch_all_fragments += score_metrics_dict[fragment]['mismatches']
        n_count_all_fragments += score_metrics_dict[fragment]['N_count']

    # pridaj parameter pre vsetky fragmenty (score = count cez vsetky fragmenty)
    score_metrics_dict['all_fragments'] = {'score': score_all_fragments, 'match': match_all_fragments, 'gaps': gaps_all_fragments, 'mismatches': mismatch_all_fragments, 'N_count':  n_count_all_fragments}

    return score_metrics_dict


In [7]:
# Zarovnanie pomocou Align.substitution_matrices
from Bio.Align import substitution_matrices


samples_score_metrics_dict_substitutionmatrix = {}

for sample_id in [samples_names[virus]]:
    
    samples_score_metrics_parameter_dict_substitutionmatrix = {}

    # cez vyber nastaveni respiracnej pipeliny
    for parameter in list(parameteres_df['parameters']):
        
        sample_name = sample_id + parameter
    
        consensus_sample_path = "./data/report_pollux/" + sample_name + "/_summary/consensus-h1n1_2019-wgs.fa"

        if os.path.exists(consensus_sample_path):

            mbs_consensus_name = mbs_consensus_names[virus]
            mbs_consensus = SeqIO.to_dict(SeqIO.parse(consensus_baseline_path, "fasta"))
            sample_consensus = SeqIO.to_dict(SeqIO.parse(consensus_sample_path, "fasta"))

            samples_score_metrics_parameter_dict_substitutionmatrix[parameter] = get_scores_for_sample_substitution_matrix(sample_consensus, mbs_consensus, mbs_consensus_name)

    samples_score_metrics_dict_substitutionmatrix[sample_id] = samples_score_metrics_parameter_dict_substitutionmatrix

samples_score_metrics_dict_substitutionmatrix

{'run230403_UVZ_BA_23-vsp-3269_': {'trimmomatic_tripq10_headcrop0_minlen35': {'PB2': {'score': 12960.0,
    'match': 2293,
    'gaps': 0,
    'mismatches': 23,
    'N_count': 17},
   'PB1': {'score': 12377.0,
    'match': 2227,
    'gaps': 0,
    'mismatches': 89,
    'N_count': 84},
   'PA': {'score': 11899.0,
    'match': 2118,
    'gaps': 0,
    'mismatches': 90,
    'N_count': 82},
   'HA': {'score': 9642.0,
    'match': 1722,
    'gaps': 0,
    'mismatches': 30,
    'N_count': 26},
   'NP': {'score': 8213.0,
    'match': 1460,
    'gaps': 0,
    'mismatches': 80,
    'N_count': 76},
   'NA': {'score': 7154.0,
    'match': 1298,
    'gaps': 0,
    'mismatches': 135,
    'N_count': 132},
   'MP': {'score': 5697.0,
    'match': 985,
    'gaps': 0,
    'mismatches': 17,
    'N_count': 29},
   'NS': {'score': 4661.0,
    'match': 829,
    'gaps': 0,
    'mismatches': 36,
    'N_count': 33},
   'all_fragments': {'score': 72603.0,
    'match': 12932,
    'gaps': 0,
    'mismatches': 500,

In [8]:
len(list(samples_score_metrics_dict_substitutionmatrix['run230403_UVZ_BA_23-vsp-3269_'].keys()))

56

In [9]:
scores_colums = ['parameters', 'score', 'match', 'gap', 'mismatch', 'N_count']
samples_score_metrics_parameter_df = pd.DataFrame(columns=scores_colums)

sample_name = 'run230403_UVZ_BA_23-vsp-3269_'

# for sample in list(samples):
for parameter in list(samples_score_metrics_dict_substitutionmatrix[sample_name].keys()):  # samples_score_metrics_dict[sample]
    score =  samples_score_metrics_dict_substitutionmatrix[sample_name][parameter]['all_fragments']['score']
    match = samples_score_metrics_dict_substitutionmatrix[sample_name][parameter]['all_fragments']['match']
    gap = samples_score_metrics_dict_substitutionmatrix[sample_name][parameter]['all_fragments']['gaps']
    mismatch = samples_score_metrics_dict_substitutionmatrix[sample_name][parameter]['all_fragments']['mismatches']
    n_count = samples_score_metrics_dict_substitutionmatrix[sample_name][parameter]['all_fragments']['N_count']


    params_df = pd.DataFrame([[parameter, score, match, gap, mismatch, n_count]], columns=scores_colums)

    samples_score_metrics_parameter_df = pd.concat([samples_score_metrics_parameter_df, params_df], axis=0, ignore_index=True)

samples_score_metrics_parameter_df

parameters    score  match gap mismatch  \
0    trimmomatic_tripq10_headcrop0_minlen35  72603.0  12932   0      500   
1    trimmomatic_tripq10_headcrop0_minlen45  72603.0  12932   0      500   
2    trimmomatic_tripq10_headcrop0_minlen55  72603.0  12932   0      500   
3    trimmomatic_tripq10_headcrop0_minlen65  72603.0  12932   0      500   
4    trimmomatic_tripq10_headcrop5_minlen35  72603.0  12932   0      500   
5    trimmomatic_tripq10_headcrop5_minlen45  72603.0  12932   0      500   
6    trimmomatic_tripq10_headcrop5_minlen65  72603.0  12932   0      500   
7   trimmomatic_tripq10_headcrop10_minlen35  72603.0  12932   0      500   
8   trimmomatic_tripq10_headcrop10_minlen45  72603.0  12932   0      500   
9   trimmomatic_tripq10_headcrop10_minlen65  72603.0  12932   0      500   
10  trimmomatic_tripq10_headcrop15_minlen45  72603.0  12932   0      500   
11  trimmomatic_tripq10_headcrop15_minlen55  72603.0  12932   0      500   
12  trimmomatic_tripq10_headcrop15_minlen65  72603.0  12932   0      500   
13   trimmomatic_tripq15_headcrop0_minlen35  72597.0  12931   0      501   
14   trimmomatic_tripq15_headcrop0_minlen45  72597.0  12931   0      501   
15   trimmomatic_tripq15_headcrop0_minlen55  72597.0  12931   0      501   
16   trimmomatic_tripq15_headcrop0_minlen65  72597.0  12931   0      501   
17   trimmomatic_tripq15_headcrop5_minlen35  72597.0  12931   0      501   
18   trimmomatic_tripq15_headcrop5_minlen45  72597.0  12931   0      501   
19   trimmomatic_tripq15_headcrop5_minlen65  72597.0  12931   0      501   
20  trimmomatic_tripq15_headcrop10_minlen45  72597.0  12931   0      501   
21  trimmomatic_tripq15_headcrop15_minlen45  72597.0  12931   0      501   
22  trimmomatic_tripq15_headcrop15_minlen55  72597.0  12931   0      501   
23   trimmomatic_tripq20_headcrop0_minlen35  72527.0  12921   0      511   
24   trimmomatic_tripq20_headcrop0_minlen45  72527.0  12921   0      511   
25   trimmomatic_tripq20_headcrop0_minlen65  72527.0  12921   0      511   
26   trimmomatic_tripq20_headcrop5_minlen35  72527.0  12921   0      511   
27   trimmomatic_tripq20_headcrop5_minlen45  72527.0  12921   0      511   
28   trimmomatic_tripq20_headcrop5_minlen65  72527.0  12921   0      511   
29  trimmomatic_tripq20_headcrop10_minlen45  72527.0  12921   0      511   
30  trimmomatic_tripq20_headcrop10_minlen55  72527.0  12921   0      511   
31  trimmomatic_tripq20_headcrop15_minlen35  72527.0  12921   0      511   
32  trimmomatic_tripq20_headcrop15_minlen45  72527.0  12921   0      511   
33  trimmomatic_tripq20_headcrop15_minlen55  72527.0  12921   0      511   
34  trimmomatic_tripq20_headcrop15_minlen65  72527.0  12921   0      511   
35   trimmomatic_tripq25_headcrop0_minlen35  72434.0  12907   0      525   
36   trimmomatic_tripq25_headcrop0_minlen45  72434.0  12907   0      525   
37   trimmomatic_tripq25_headcrop0_minlen55  72434.0  12907   0      525   
38   trimmomatic_tripq25_headcrop0_minlen65  72434.0  12907   0      525   
39   trimmomatic_tripq25_headcrop5_minlen35  72434.0  12907   0      525   
40   trimmomatic_tripq25_headcrop5_minlen55  72434.0  12907   0      525   
41   trimmomatic_tripq25_headcrop5_minlen65  72434.0  12907   0      525   
42  trimmomatic_tripq25_headcrop10_minlen35  72434.0  12907   0      525   
43  trimmomatic_tripq25_headcrop10_minlen45  72434.0  12907   0      525   
44  trimmomatic_tripq25_headcrop10_minlen55  72434.0  12907   0      525   
45  trimmomatic_tripq25_headcrop10_minlen65  72434.0  12907   0      525   
46  trimmomatic_tripq25_headcrop15_minlen35  72434.0  12907   0      525   
47  trimmomatic_tripq25_headcrop15_minlen45  72434.0  12907   0      525   
48  trimmomatic_tripq25_headcrop15_minlen55  72434.0  12907   0      525   
49  trimmomatic_tripq25_headcrop15_minlen65  72434.0  12907   0      525   
50   trimmomatic_tripq30_headcrop0_minlen35  68122.0  12281   0     1151   
51   trimmomatic_tripq30_headcrop0_minlen45  68122.0  12281   0     1151   
52   

In [10]:
import numpy as np

col_names_param = ['trimmer', 'headcrop', 'quality', 'score', 'match', 'gap', 'mismatch', 'N_count']

scores_df = pd.DataFrame(columns=col_names_param)

for _, row in samples_score_metrics_parameter_df.iterrows():
    
    row_scores_df = pd.DataFrame([[np.nan, np.nan, np.nan, row['score'], row['match'], row['gap'], row['mismatch'], row['N_count']]],
                                   columns=col_names_param)
    param_name = row['parameters']
    
    row_scores_df['trimmer'] = param_name.split('_')[0]
    row_scores_df['headcrop'] = param_name.split('_')[1].replace('tripq', '')
    row_scores_df['quality'] = param_name.split('_')[2].replace('headcrop', '')

    scores_df = pd.concat([scores_df, row_scores_df], axis=0, ignore_index=True)

scores_df


trimmer headcrop quality    score  match gap mismatch N_count
0   trimmomatic       10       0  72603.0  12932   0      500     479
1   trimmomatic       10       0  72603.0  12932   0      500     479
2   trimmomatic       10       0  72603.0  12932   0      500     479
3   trimmomatic       10       0  72603.0  12932   0      500     479
4   trimmomatic       10       5  72603.0  12932   0      500     479
5   trimmomatic       10       5  72603.0  12932   0      500     479
6   trimmomatic       10       5  72603.0  12932   0      500     479
7   trimmomatic       10      10  72603.0  12932   0      500     479
8   trimmomatic       10      10  72603.0  12932   0      500     479
9   trimmomatic       10      10  72603.0  12932   0      500     479
10  trimmomatic       10      15  72603.0  12932   0      500     479
11  trimmomatic       10      15  72603.0  12932   0      500     479
12  trimmomatic       10      15  72603.0  12932   0      500     479
13  trimmomatic       15       0  72597.0  12931   0      501     480
14  trimmomatic       15       0  72597.0  12931   0      501     480
15  trimmomatic       15       0  72597.0  12931   0      501     480
16  trimmomatic       15       0  72597.0  12931   0      501     480
17  trimmomatic       15       5  72597.0  12931   0      501     480
18  trimmomatic       15       5  72597.0  12931   0      501     480
19  trimmomatic       15       5  72597.0  12931   0      501     480
20  trimmomatic       15      10  72597.0  12931   0      501     480
21  trimmomatic       15      15  72597.0  12931   0      501     480
22  trimmomatic       15      15  72597.0  12931   0      501     480
23  trimmomatic       20       0  72527.0  12921   0      511     490
24  trimmomatic       20       0  72527.0  12921   0      511     490
25  trimmomatic       20       0  72527.0  12921   0      511     490
26  trimmomatic       20       5  72527.0  12921   0      511     490
27  trimmomatic       20       5  72527.0  12921   0      511     490
28  trimmomatic       20       5  72527.0  12921   0      511     490
29  trimmomatic       20      10  72527.0  12921   0      511     490
30  trimmomatic       20      10  72527.0  12921   0      511     490
31  trimmomatic       20      15  72527.0  12921   0      511     490
32  trimmomatic       20      15  72527.0  12921   0      511     490
33  trimmomatic       20      15  72527.0  12921   0      511     490
34  trimmomatic       20      15  72527.0  12921   0      511     490
35  trimmomatic       25       0  72434.0  12907   0      525     504
36  trimmomatic       25       0  72434.0  12907   0      525     504
37  trimmomatic       25       0  72434.0  12907   0      525     504
38  trimmomatic       25       0  72434.0  12907   0      525     504
39  trimmomatic       25       5  72434.0  12907   0      525     504
40  trimmomatic       25       5  72434.0  12907   0      525     504
41  trimmomatic       25       5  72434.0  12907   0      525     504
42  trimmomatic       25      10  72434.0  12907   0      525     504
43  trimmomatic       25      10  72434.0  12907   0      525     504
44  trimmomatic       25      10  72434.0  12907   0      525     504
45  trimmomatic       25      10  72434.0  12907   0      525     504
46  trimmomatic       25      15  72434.0  12907   0      525     504
47  trimmomatic       25      15  72434.0  12907   0      525     504
48  trimmomatic       25      15  72434.0  12907   0      525     504
49  trimmomatic       25      15  72434.0  12907   0      525     504
50  trimmomatic       30       0  68122.0  12281   0     1151    1132
51  trimmomatic       30       0  68122.0  12281   0     1151    1132
52  trimmomatic       30       0  68122.0  12281   0     1151    1132
53  trimmomatic       30       0  68122.0  12281   0     1151    1132
54  trimmomatic       30       5  68122.0  12281   0     1151    1132
55  trimmomatic       30       5  68122.0  12281   0     1151    1132

In [11]:
scores_df.to_csv('./data/result_metrics/sample_run230403_UVZ_BA_23-vsp-3269_scores_blosum.tsv', sep='\t', index=False)
